In [ ]:
import pandas as pd
import requests
import json
from pandas_datareader import data

import pickle


In [ ]:
ticker = 'TSLA'


In [ ]:
si_data = requests.get(f'https://api.nasdaq.com/api/quote/{ticker}/short-interest?assetClass=stocks',
                    headers={"Accept-Language": "en-US,en;q=0.9",
                             "Accept-Encoding": "gzip, deflate, br",
                             "User-Agent": "Java-http-client/"}).text


In [ ]:
price_data = data.DataReader(ticker, 'yahoo', '2020-10-01', '2021-10-03')
price_data

In [ ]:
data_json = json.loads(si_data)
si_df = pd.json_normalize(data_json, record_path=['data','shortInterestTable', 'rows'])
si_df['date'] = pd.to_datetime(si_df['settlementDate'])
si_df

In [ ]:
ibd_content = requests.get(f'https://iborrowdesk.com/api/ticker/{ticker}').text


In [ ]:
ibd_json = json.loads(ibd_content)
ibd_df = pd.json_normalize(ibd_json, record_path=['daily'])
ibd_df['date'] = pd.to_datetime(ibd_df['time'])
ibd_df

In [ ]:
from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits import axisartist
import matplotlib.pyplot as plt

host = host_subplot(111, axes_class=axisartist.Axes,figure=plt.figure(figsize=(30,10)))
plt.subplots_adjust(right=0.75)


par1 = host.twinx()
par2 = host.twinx()

par2.axis["right"] = par2.new_fixed_axis(loc="right", offset=(60, 0))

par1.axis["right"].toggle(all=True)
par2.axis["right"].toggle(all=True)

p1, = host.plot(price_data.index, price_data['Adj Close'], label="Adj Close")
p2, = par1.plot(si_df['date'], si_df['daysToCover'], label="Short interest")
p3, = par2.plot(ibd_df['date'], ibd_df['fee'], label="Borrowing free")


host.set_xlabel("Time")
host.set_ylabel("Price")
par1.set_ylabel("Short interest")
par2.set_ylabel("Borrowing fee")

host.legend()

host.axis["left"].label.set_color(p1.get_color())
par1.axis["right"].label.set_color(p2.get_color())
par2.axis["right"].label.set_color(p3.get_color())

plt.show()

# Save and load functions

In [ ]:
# price_dict = {}
# si_dict = {}
# ibd_dict = {}

In [ ]:
with open('price_dict.pkl', 'rb') as f:
    price_dict = pickle.load(f)
    
with open('si_dict.pkl', 'rb') as f:
    si_dict = pickle.load(f)

with open('ibd_dict.pkl', 'rb') as f:
    ibd_dict = pickle.load(f)

In [ ]:
price_dict[ticker] = price_data
si_dict[ticker] = si_df
ibd_dict[ticker] = ibd_df

In [ ]:
with open('price_dict.pkl', 'wb') as f:
    pickle.dump(price_dict, f, pickle.HIGHEST_PROTOCOL)

with open('si_dict.pkl', 'wb') as f:
    pickle.dump(si_dict, f, pickle.HIGHEST_PROTOCOL)

with open('ibd_dict.pkl', 'wb') as f:
    pickle.dump(ibd_dict, f, pickle.HIGHEST_PROTOCOL)

# Other stuff

In [ ]:

price_data['Adj Close'].plot(figsize=(30, 5))
si_df.plot(x='date', figsize=(30, 5))
ibd_df.plot(x='date', y='fee', figsize=(30, 5))


In [ ]:
avg_borrowing_fee = ibd_df['fee'].mean()
print(f"Average borrowing fee: {avg_borrowing_fee}")
avg_si_ratio = si_df['daysToCover'].mean()
print(f"Average short interest ratio: {avg_si_ratio}")


In [ ]:
ret_series = (1 + price_data['Adj Close'].pct_change()).cumprod() - 1
annual_return_row = ret_series.tail(1)
annual_return = annual_return_row[0]

In [ ]:
# only uncomment when creating new data
# store = pd.DataFrame(None,columns=['Average borrowing fee', 'Average short interest ratio', 'Annual return'])

In [ ]:
store.loc[ticker] = pd.Series([avg_borrowing_fee, avg_si_ratio, annual_return], index=store.columns)

store

In [ ]:
store.to_csv('store.csv')

In [ ]:
store.plot.scatter('Average borrowing fee', 'Annual return')

In [ ]:
store.plot.scatter('Average short interest ratio', 'Annual return')